In [27]:
import sys
import os
import datetime
import calendar
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib.dates import YearLocator, MonthLocator, DayLocator, HourLocator, MinuteLocator, DateFormatter
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
import omni_utils
from pathlib import Path

In [2]:
doPlots = 1

In [3]:
# for a in os.environ:
#     print(a)
# sys.path.insert(0,'/home/spencerh/python/hatch_python_utils/')
# #sys.path
# print(sys.path)

In [4]:
omni = omni_utils.load_omniDB()

Opening culled_OMNI_magdata.dat ...


--------------------------------------------------
Date: Fri May  6 16:53:37 2016
User: spencerh
Host: thelonious.dartmouth.edu
--------------------------------------------------
Format: 12
Architecture: x86_64
Operating System: linux
IDL Version: 8.4
--------------------------------------------------
Successfully read 20 records of which:
 - 17 are of type VARIABLE
 - 1 are of type TIMESTAMP
 - 1 are of type VERSION
--------------------------------------------------
Available variables:
 - cone_overclock_gsm [<class 'numpy.ndarray'>]
 - mag_utc [<class 'numpy.ndarray'>]
 - bz_gse [<class 'numpy.ndarray'>]
 - bxy_over_bz_gsm [<class 'numpy.ndarray'>]
 - thetacone_gsm [<class 'numpy.ndarray'>]
 - by_gse [<class 'numpy.ndarray'>]
 - bz_gsm [<class 'numpy.ndarray'>]
 - bt_gse [<class 'numpy.ndarray'>]
 - bxy_over_bz_gse [<class 'numpy.ndarray'>]
 - goodmag_goodtimes_i [<class 'numpy.ndarray'>]
 - by_gsm [<class 'numpy.ndarray'>]
 - bt_gsm [<class 'numpy.ndarray'>]
 - cone_overclock_gse [<

In [5]:
# # As seconds since 1970:
# print(omni.mag_utc[0])
# # As nicely formatted thing:
# print(datetime.datetime.fromtimestamp(omni.mag_utc[0]))
# # Convert from datetime to UTC
# print(datetime.datetime.fromtimestamp(omni.mag_utc[0],tz=datetime.timezone.utc).replace(tzinfo=datetime.timezone.utc).timestamp())
# # And then back
# print(datetime.datetime.fromtimestamp(838857600.0))

In [22]:
# Read in the ol' txtFile
kappaTxtFile='/SPENCEdata/Research/Satellites/FAST/espec_identification/txtOutput/20180302--mono_aurora_streaks__-5-5MLT__1000-3999ORB__300-4300ALT__minTStreak_sec_60.txt'
kappaFmt=np.dtype([('tmpOrb',np.int32),('MLT',np.float64),('ILAT',np.float64),('ALT',np.float64),('t1Str',np.unicode_,19),('t2Str',np.unicode_,8),('t_streakLen',np.int32),('nPts',np.int32),('dt_avg',np.float64),('avg_current',np.float64)])
kappaInfo = np.genfromtxt(kappaTxtFile,dtype=kappaFmt,skip_header=1)
kappaRec = np.rec.array(kappaInfo)
kappaRec.sort(order='tmpOrb')   # Sort so that plots are ordered by orbit number

In [23]:
t2Str = kappaRec.t2Str
t1Str = kappaRec.t1Str

# See if we can do substrings
# this = [x[0:11] for x in t1Str]
# print(this)
#this?

# Yup, now combine 'em all
this = ["{}{}".format(t1_[0:11],t2_) for t1_, t2_ in zip(t1Str,t2Str)]

# Print 'em for kicks
# for t1, t2 in zip(t1Str,this):
#     print(t1,", ",t2)

# And convert to datetime objects
t1s=[datetime.datetime.strptime(t1,'%Y-%m-%d/%H:%M:%S') for t1 in t1Str]
t2s=[datetime.datetime.strptime(t2,'%Y-%m-%d/%H:%M:%S') for t2 in this]

# Does it all work? Looks like the answer is 'yes'
# for t1,t2,bros in zip(t1s,t2s,kappaRec.t_streakLen):
#     print(t1,",",t2,",",t2-t1,",",bros)

# Times before and after
# tIncrem=datetime.timedelta(hours=1)
# for t1 in t1s:
#    print(t1-tIncrem,", ",t1+tIncrem)


In [24]:
#t1s.replace(tzinfo=datetime.timezone.utc)
t1Upp = [t1.replace(tzinfo=datetime.timezone.utc) for t1 in t1s]
t2Upp = [t2.replace(tzinfo=datetime.timezone.utc) for t2 in t2s]
t1s   = t1Upp
t2s   = t2Upp

t1UTC = [t1.timestamp() for t1 in t1s]
t2UTC = [t2.timestamp() for t2 in t2s]

In [25]:
########################################
# Set up plots
########################################

# Set font sizes for things
legFontSize = 20
tickFontSize=25
titleFontSize=30
matplotlib.rc('xtick',labelsize=tickFontSize)
matplotlib.rc('ytick',labelsize=tickFontSize)
matplotlib.rc('axes',titlesize=titleFontSize)

# Set up ticks
ymajorLocator = MultipleLocator(1)
yminorLocator = MultipleLocator(0.25)

xAxisHours = MinuteLocator(interval=30)   # every hour
xAxisMinutes = MinuteLocator(interval=5)  # every minute
xAxisHoursFmt = DateFormatter('%H:%M')

In [53]:
########################################
# Test a plot
########################################
tIncremSeconds = 60*60
outDir='/SPENCEdata/Research/Satellites/FAST/kappa_dists/plots/20180314/'
outFPref = 'midnight_kappa__orb'
outFSuff = '__SW_conds.png'

checkForExistingPlots = 1
# k=0
if 'doPlots' in globals() and doPlots == 1:
    for k in range(0,kappaRec.tmpOrb.size-1):

        # tmpInds = ((omni.mag_utc > (t1UTC[k]-tIncremSeconds)) & (omni.mag_utc < (t2UTC[k]+tIncremSeconds)))
        # print(omni.mag_utc[tmpInds].size)
        outFName='{:02}__'.format(k)+outFPref+'{}'.format(kappaRec.tmpOrb[k])+outFSuff
        if 'checkForExistingPlots' in globals() and checkForExistingPlots == 1:
            tmpPath=Path(outDir+outFName)
            if tmpPath.exists():
                print("Already have {} ... Skipping!".format(outFName))
                continue
        tmpIndStart=np.argmin(np.abs(omni.mag_utc-(t1UTC[k]-tIncremSeconds)))
        tmpIndStop=np.argmin(np.abs(omni.mag_utc-(t2UTC[k]+tIncremSeconds)))
        print("[{:02}] Orbit {} (inds {}:{})".format(k,kappaRec.tmpOrb[k],tmpIndStart,tmpIndStop))
        if tmpIndStart == tmpIndStop or (tmpIndStop-tmpIndStart) == 1 or (tmpIndStop < tmpIndStart):
            print("Can't do it!")
            continue
        bx=omni.bx[tmpIndStart:tmpIndStop]
        by=omni.by_gsm[tmpIndStart:tmpIndStop]
        bz=omni.bz_gsm[tmpIndStart:tmpIndStop]
        time=[datetime.datetime.fromtimestamp(tid,tz=datetime.timezone.utc) for tid in omni.mag_utc[tmpIndStart:tmpIndStop]]
        dates = matplotlib.dates.date2num(time)
        print("Time[0]: {}".format(time[0]))
        fig, ax = plt.subplots(figsize=(20,13))

        # Plot the data
        ax.plot_date(dates, bx, '-',linewidth=3)
        ax.plot_date(dates, by, '-',linewidth=3)
        ax.plot_date(dates, bz, '-',linewidth=3)
    
        # format the ticks
        ax.yaxis.set_minor_locator(yminorLocator)
        ax.yaxis.set_major_locator(ymajorLocator)

        ax.xaxis.set_major_locator(xAxisHours)
        ax.xaxis.set_major_formatter(xAxisHoursFmt)
        ax.xaxis.set_minor_locator(xAxisMinutes)
        # ax.set_xlim(time[0], time[-1])
        # ax.autoscale_view()

        # Grid and legend
        ax.grid(which='minor',axis='both')
        ax.legend(['Bx','By (GSM)','Bz (GSM)'],fontsize=legFontSize)

        # Title
        ax.set_title('Orbit {} ('.format(kappaRec.tmpOrb[k])+time[0].strftime('%Y-%m-%d')+')')
        ax.axvline(x=t1s[k],linestyle='dashed',linewidth=2.1,color='r')
        ax.axvline(x=t2s[k],linestyle='dashed',linewidth=2.1,color='r')
        print("Saving to {} ...".format(outFName))
        plt.savefig(outDir+outFName)
        plt.close()

Already have 00__midnight_kappa__orb1182__SW_conds.png ... Skipping!
Already have 01__midnight_kappa__orb1257__SW_conds.png ... Skipping!
Already have 02__midnight_kappa__orb1259__SW_conds.png ... Skipping!
Already have 03__midnight_kappa__orb1450__SW_conds.png ... Skipping!
Already have 04__midnight_kappa__orb1504__SW_conds.png ... Skipping!
Already have 05__midnight_kappa__orb1562__SW_conds.png ... Skipping!
Already have 06__midnight_kappa__orb1563__SW_conds.png ... Skipping!
Already have 07__midnight_kappa__orb1567__SW_conds.png ... Skipping!
Already have 08__midnight_kappa__orb1569__SW_conds.png ... Skipping!
Already have 09__midnight_kappa__orb1576__SW_conds.png ... Skipping!
Already have 10__midnight_kappa__orb1578__SW_conds.png ... Skipping!
Already have 11__midnight_kappa__orb1579__SW_conds.png ... Skipping!
Already have 12__midnight_kappa__orb1607__SW_conds.png ... Skipping!
Already have 13__midnight_kappa__orb1607__SW_conds.png ... Skipping!
Already have 14__midnight_kappa__o

[55] Orbit 1822 (inds 240880:240987)
Time[0]: 1997-02-05 21:14:00+00:00
Saving to 55__midnight_kappa__orb1822__SW_conds.png ...


[56] Orbit 1829 (inds 241768:241889)
Time[0]: 1997-02-06 12:51:00+00:00


Saving to 56__midnight_kappa__orb1829__SW_conds.png ...


[57] Orbit 1835 (inds 242535:242626)
Time[0]: 1997-02-07 02:02:00+00:00
Saving to 57__midnight_kappa__orb1835__SW_conds.png ...


...

[190] Orbit 3918 (inds 497981:498095)
Time[0]: 1997-08-18 17:28:00+00:00
Saving to 190__midnight_kappa__orb3918__SW_conds.png ...


[191] Orbit 3931 (inds 499488:499608)
Time[0]: 1997-08-19 22:11:00+00:00
Saving to 191__midnight_kappa__orb3931__SW_conds.png ...


[192] Orbit 3942 (inds 500934:501045)
Time[0]: 1997-08-20 22:35:00+00:00
Saving to 192__midnight_kappa__orb3942__SW_conds.png ...


[193] Orbit 3953 (inds 502369:502488)
Time[0]: 1997-08-21 22:57:00+00:00
Saving to 193__midnight_kappa__orb3953__SW_conds.png ...


[194] Orbit 3986 (inds 506743:506865)
Time[0]: 1997-08-25 00:09:00+00:00
Saving to 194__midnight_kappa__orb3986__SW_conds.png ...


In [33]:
?time

In [184]:
tIncremSeconds = 60*60
for k in range(0,kappaRec.tmpOrb.size-1):
    print("Orbit {}".format(kappaRec.tmpOrb[k]))
    # tmpInds = ((omni.mag_utc > (t1UTC[k]-tIncremSeconds)) & (omni.mag_utc < (t2UTC[k]+tIncremSeconds)))
    # print(omni.mag_utc[tmpInds].size)
    # tmpIndStart=np.argmin(np.abs(omni.mag_utc-(t1UTC[k]-tIncremSeconds)))
    # tmpIndStop=np.argmin(np.abs(omni.mag_utc-(t2UTC[k]+tIncremSeconds)))
#    print("{}:{}".format(tmpIndStart,tmpIndStop))

Orbit 2585
Orbit 1217
Orbit 1368
Orbit 2487
Orbit 2597
Orbit 1607
Orbit 1294
Orbit 2640
Orbit 2347
Orbit 1152
Orbit 2379
Orbit 3956
Orbit 2586
Orbit 2390
Orbit 2685
Orbit 1259
Orbit 3741
Orbit 1940
Orbit 1272
Orbit 2702
Orbit 1232
Orbit 3848
Orbit 2446
Orbit 2381
Orbit 1182
Orbit 2822
Orbit 3846
Orbit 1562
Orbit 1855
Orbit 1758
Orbit 3592
Orbit 1433
Orbit 3991
Orbit 3783
Orbit 1293
Orbit 3785
Orbit 1152
Orbit 1991
Orbit 1952
Orbit 3837
Orbit 1238
Orbit 3815
